# Base de datos neo4j

## Lectura de datos

In [1]:
import pandas as pd

df_dificultades = pd.read_csv("./data/dificultades.csv",encoding='utf-8');
df_encadenamientos = pd.read_csv("./data/tipos_encadenamiento.csv", encoding='utf-8');
df_escaladores = pd.read_csv("./data/escaladores_lite_2017.csv", encoding='utf-8');
df_ascensos = pd.read_csv("./data/ascensos_lite_2017.csv", encoding='utf-8');

## Conexión con neo4j y borrado de BB.DD.

In [2]:
%load_ext cypher

In [3]:
from py2neo import Graph, Node, Relationship

graph = Graph()

In [4]:
# Borrado de todos los nodos y relaciones
graph.delete_all()

## Carga de datos en neo4j

 El modelo de nodos y relaciones que voy a considerar es este:
 
![NODOS y RELACIONES](./img/nodosRelacionesNEO.jpg)
 
 
 

Desnormalizo los datos de ascensos para facilitar la introducción de nodos y relaciones en la base de datos

In [5]:
df_ascensos = pd.merge(df_ascensos, df_encadenamientos, on = ['id_tipo_encadenamiento'], how = 'inner')
df_ascensos = pd.merge(df_ascensos, df_dificultades, on = ['id_dificultad'], how = 'inner');
df_ascensos = pd.merge(df_ascensos, df_escaladores, on = ['id_escalador'], how = 'inner');

# Renombrar campos por claridad (el merge auto-asigna nombres a campos repetidos)
df_ascensos = df_ascensos.rename(
    columns={
        'nombre': 'nombre_escalador',
        'ciudad':'ciudad_escalador',
        'pais_y': 'pais_escalador',
        'comienzo': 'anio_comienzo',
        'pais_x': 'pais_risco'});

In [6]:
df_ascensos.head(2)

,id_escalador,id_dificultad,id_tipo_encadenamiento,nombre_via,risco,sector,fecha,pais_risco,tipo_encadenamiento,grado_frances,nombre_escalador,sexo,fecha_nacimiento,ciudad_escalador,pais_escalador,anio_comienzo
0,20384,33,2,LE DE,FONTAINEBLEAU,ROCHER CANON,2017-04-17,FRA,Flash,5c,Clemens Kurth,Hombre,1982-04-24,Alfeld (Leine),DEU,2003
1,20384,33,2,SCHMALSEITE DV,ELBSANDSTEIN,KRALLENTURM,2017-06-10,DEU,Flash,5c,Clemens Kurth,Hombre,1982-04-24,Alfeld (Leine),DEU,2003


Convierto el data frame de ascensos desnormalizado a JSON para la inserción de información en neo4j

In [7]:
import json
json_string = df_ascensos.to_json(orient = 'records')
json_list = json.loads(json_string)

Iterando en el data frame de ascensos, voy añadiendo los nodos y las relaciones que se generan por cada ascenso:

In [8]:
for index, ascenso_json in enumerate(json_list):
    # Nodos
    escalador = Node("Escalador", id=ascenso_json["id_escalador"],
                                  nombre = ascenso_json["nombre_escalador"],
                                  sexo = ascenso_json["sexo"],
                                  pais = ascenso_json["pais_escalador"],
                                  anio_comienzo = ascenso_json["anio_comienzo"])   
    
    ascenso = Node("Ascenso", nombre=ascenso_json["nombre_via"], 
                              tipo_encadenamiento = ascenso_json["tipo_encadenamiento"],
                              risco = ascenso_json["risco"],
                              pais = ascenso_json["pais_risco"]) 
    
    dificultad = Node("Dificultad", valor = ascenso_json["id_dificultad"], grado_frances = ascenso_json["grado_frances"])

    
  
        
    # Relaciones  
    rel_escalador_ascenso = Relationship(escalador, "REALIZA", ascenso, tipo_encadenamiento = ascenso_json["tipo_encadenamiento"])
    rel_ascenso_dificultad = Relationship(ascenso, "TIENE", dificultad)
    
    graph.merge(escalador|dificultad)
    graph.create(ascenso|rel_ascenso_dificultad|rel_escalador_ascenso)
    
    if index % 500 == 0:
        print index

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500


Además, como voy a tener que hacer calculos de dificultad media, necesito que los todos los nodos dificultad existan, aunque no hayan existido ascensos con esa dificultad. Por ello, los añado siguiendo una estrategia de unicidad (merge) iterando sobre el data frame de dificultades:

In [9]:
import json
json_string = df_dificultades.to_json(orient = 'records')
json_list = json.loads(json_string)

for index, dificultad_json in enumerate(json_list):
    dificultad = Node("Dificultad", valor = dificultad_json["id_dificultad"],
                                    grado_frances = dificultad_json["grado_frances"])
    graph.merge(dificultad)
    
    if index % 10 == 0:
        print index

0
10
20
30
40
50
60
70
80


### Comprobaciones previas ...

Número de escaladores

In [10]:
%%cypher
MATCH (:Escalador)
RETURN count(*)

1 rows affected.


count(*)
4580


Número de vías

In [11]:
%%cypher
MATCH (:Ascenso)
RETURN count(*)

1 rows affected.


count(*)
11832


Número de riscos

In [12]:
%%cypher
MATCH (:Dificultad)
RETURN count(*)

1 rows affected.


count(*)
86


# RESPUESTAS A LAS PREGUNTAS

### 1.a) Los 10 escaladores (hombres) más activos (orden auxiliar por Id)

In [13]:
%%cypher
MATCH (escalador:Escalador)-[:REALIZA]->(ascenso:Ascenso)
WHERE escalador.sexo = "Hombre"
RETURN escalador.id, escalador.nombre, escalador.sexo, escalador.pais, count(ascenso) as numero_ascensos
ORDER BY numero_ascensos DESC, escalador.id DESC
LIMIT 10

10 rows affected.


escalador.id,escalador.nombre,escalador.sexo,escalador.pais,numero_ascensos
50884,Christopher Leonetti,Hombre,USA,47
20384,Clemens Kurth,Hombre,DEU,26
20095,Matthias Schuster,Hombre,DEU,26
66250,Kuba Kaminski,Hombre,POL,24
9171,Laurenz Trawnicek,Hombre,AUT,23
46622,Raúl Crespo,Hombre,ESP,19
42086,Marcin Opozda,Hombre,POL,19
66466,Christian Boehme,Hombre,DEU,18
47732,Thomas de Fleurian,Hombre,FRA,18
35847,Zack Bum,Hombre,DEU,18


### 1.b) Los 10 escaladoras (mujeres) más activas (orden auxiliar por Id)

In [14]:
%%cypher
MATCH (escalador:Escalador)-[:REALIZA]->(ascenso:Ascenso)
WHERE escalador.sexo = "Mujer"
RETURN escalador.id, escalador.nombre, escalador.pais, count(ascenso) as numero_ascensos
ORDER BY numero_ascensos DESC, escalador.id DESC
LIMIT 10

10 rows affected.


escalador.id,escalador.nombre,escalador.pais,numero_ascensos
65502,Ksenia Targosz,POL,13
65069,Mania Mania A.,POL,12
65707,Ewelina Cienkus,POL,11
54695,Elfi Hasler,AUT,11
62354,La Shoune,FRA,10
58835,Dominika Sołtys,POL,10
57447,Sue Murphy,CAN,10
53983,Daniela Bärtschi,CHE,10
49569,Karina Kosiorek,POL,10
32043,Reidun M. Romundstad,NOR,10


### 2. Lista de los 10 ascensos "On sight" de la escaladora más activa en orden decreciente de dificultad (y por nombre de via ascendete)

In [18]:
%%cypher

MATCH (escalador:Escalador)-[:REALIZA]-(ascenso:Ascenso)
WHERE escalador.sexo = "Mujer"
WITH  escalador, count(ascenso) as numero_ascensos
ORDER BY numero_ascensos DESC, escalador.id DESC
LIMIT 1

MATCH (escalador:Escalador)-[:REALIZA]-(asc:Ascenso)-[:TIENE]-(dif:Dificultad)
WHERE asc.tipo_encadenamiento = 'Onsight'
RETURN asc.nombre, dif.grado_frances, asc.risco, asc.pais
ORDER BY dif.valor DESC, asc.nombre ASC

8 rows affected.


asc.nombre,dif.grado_frances,asc.risco,asc.pais
LEWIZNA,5c,DOLINA KOBYLANSKA,POL
MOJA KOKAINA,5c,DOLINA KOBYLANSKA,POL
RUCHY MASOWE,5c,DOLINA BEDKOWSKA,POL
FISCHER-LAPINSKI,5b,DOLINA BEDKOWSKA,POL
GRZBIET ZUBRA,5b,SOKOLIKI,POL
RYSKOWIEC,5b,DOLINA BEDKOWSKA,POL
ZAGłADA TRADA,4c,DOLINA KOBYLANSKA,POL
KIKUT,4b,DOLINA SZKLARKI,POL


### 3. Dificultad media de los ascensos del escalador más activo

In [20]:
%%cypher

MATCH (escalador:Escalador)-[:REALIZA]-(ascenso:Ascenso)
WHERE escalador.sexo = "Hombre" 
WITH escalador, count(ascenso) as num_ascensos
ORDER BY num_ascensos DESC
LIMIT 1

MATCH (escalador:Escalador)-[:REALIZA]-(:Ascenso)-[:TIENE]-(dif:Dificultad)
WITH round(avg(dif.valor)) as valor_dif_media

MATCH (dif_media:Dificultad {valor : valor_dif_media})
RETURN dif_media.grado_frances

1 rows affected.


dif_media.grado_frances
5c+


### 4.a) Los 10 ascensos mas dificiles

In [24]:
%%cypher
MATCH (escalador:Escalador)-[:REALIZA]-(ascenso:Ascenso)-[:TIENE]-(dif:Dificultad)
RETURN escalador.nombre,
       escalador.pais,
       ascenso.nombre,
       dif.grado_frances,
       ascenso.tipo_encadenamiento,
       ascenso.risco,
       ascenso.pais
ORDER BY dif.valor DESC
LIMIT 10

10 rows affected.


escalador.nombre,escalador.pais,ascenso.nombre,dif.grado_frances,ascenso.tipo_encadenamiento,ascenso.risco,ascenso.pais
Adam Ondra,CZE,SILENCE,9c,Redpoint,FLATANGER,NOR
Stefano Ghisolfi,ITA,FIRST ROUND FIRST MINUTE,9b,Redpoint,MARGALEF,ESP
Adam Ondra,CZE,MOVE HARD,9b,Redpoint,FLATANGER,NOR
Stefano Ghisolfi,ITA,ONE PUNCH,9a+,Redpoint,ARCO,ITA
Stefano Ghisolfi,ITA,LA RAMBLA,9a+,Redpoint,SIURANA,ESP
David Firnenburg,DEU,LA RAMBLA,9a+,Redpoint,SIURANA,ESP
Piotr Schab,POL,THOR'S HAMMER,9a+,Redpoint,FLATANGER,NOR
Stefano Ghisolfi,ITA,FIRST LEY,9a+,Redpoint,MARGALEF,ESP
Evan Hau,CAN,HONOUR AND GLORY,9a+,Redpoint,ECHO CANYON,CAN
Daniel Fuertes,ESP,NO PAIN NO GAIN,9a+,Redpoint,RODELLAR,ESP


### 4.b) Los 10 ascensos mas dificiles a vista (On sight)

In [25]:
%%cypher
MATCH (escalador:Escalador)-[:REALIZA]-(ascenso:Ascenso)-[:TIENE]-(dif:Dificultad)
WHERE ascenso.tipo_encadenamiento='Onsight'
RETURN escalador.nombre,
       escalador.pais,
       ascenso.nombre,
       dif.grado_frances,
       ascenso.tipo_encadenamiento,
       ascenso.risco,
       ascenso.pais
ORDER BY dif.valor DESC
LIMIT 10

10 rows affected.


escalador.nombre,escalador.pais,ascenso.nombre,dif.grado_frances,ascenso.tipo_encadenamiento,ascenso.risco,ascenso.pais
Stefano Ghisolfi,ITA,FISH EYE,8c,Onsight,OLIANA,ESP
Piotr Schab,POL,PEQUENA ESTRELLA,8b+,Onsight,RODELLAR,ESP
Manu Lopez,FRA,WHAT,8b,Onsight,LEONIDIO,GRC
luis rodriguez martin,ESP,BRUJO,8b,Onsight,SADERNES,ESP
Adam Ondra,CZE,MATA HARI,8a+,Onsight,FRANKENJURA,DEU
Dmitrii Fakirianov,RUS,L'ADRECADOR,8a+,Onsight,TERRADETS,ESP
Michaela Kiersch,USA,LA FEMME BLANCHE,8a+,Onsight,CéüSE,FRA
jose luis palao,ESP,TEAM BTR,8a+,Onsight,BIELSA,ESP
jose luis palao,ESP,REALIDAD VIRTUAL,8a+,Onsight,POLORIA,ESP
Marcin Wszolek,POL,NUEVE ZETA,8a+,Onsight,CHULILLA,ESP


### 5.a) Grado medio y maximo de los ascensos en España de los 10 escaladores NO ESPAÑOLES con mas ascensos en España

In [9]:
%%cypher 

MATCH (escalador:Escalador)-[:REALIZA]-(ascenso:Ascenso)-[:TIENE]-(dif:Dificultad)
WHERE escalador.pais <> 'ESP' AND ascenso.pais = 'ESP'
WITH escalador, round(avg(dif.valor)) as dif_valor_medio, max(dif.valor) as dif_valor_max, count(ascenso) as num_ascensos
ORDER BY num_ascensos DESC, escalador.id ASC
LIMIT 10

MATCH (dif_media:Dificultad{valor:dif_valor_medio}), (dif_max:Dificultad{valor:dif_valor_max})
RETURN escalador.nombre, escalador.pais,
       num_ascensos,
       dif_media.grado_frances as grado_medio,
       dif_max.grado_frances as grado_maximo





10 rows affected.


escalador.nombre,escalador.pais,num_ascensos,grado_medio,grado_maximo
Nuno Henriques,PRT,14,6a+,7a
"Grzegorz ""Buła"" Golowczyk",POL,9,7b+,8b+
philipp kieffer,DEU,7,6c+/7a,7c
Wojtek Pełka,POL,7,7c+/8a,8b
"Gonçalo ""Gongas"" Coutinho",PRT,6,5a,6a
Tieme van Veen,NLD,6,6c+/7a,8a
Kuba Pe,POL,6,7a,7b+
Gabriel Korbiel,POL,6,7a+/7b,7c
Amber Thornton,GBR,6,5c,6c
Benjamin Thomas,FRA,6,7a,7c+


### 5.b) Grado medio y maximo de los ascensos en España de los 10 escaladores ESPAÑOLES con mas ascensos en España

In [10]:
%%cypher 

MATCH (escalador:Escalador)-[:REALIZA]-(ascenso:Ascenso)-[:TIENE]-(dif:Dificultad)
WHERE escalador.pais = 'ESP' AND ascenso.pais = 'ESP'
WITH escalador, round(avg(dif.valor)) as dif_valor_medio, max(dif.valor) as dif_valor_max, count(ascenso) as num_ascensos
ORDER BY num_ascensos DESC, escalador.id ASC
LIMIT 10

MATCH (dif_media:Dificultad{valor:dif_valor_medio}), (dif_max:Dificultad{valor:dif_valor_max})
RETURN escalador.nombre, escalador.pais,
       num_ascensos,
       dif_media.grado_frances as grado_medio,
       dif_max.grado_frances as grado_maximo

10 rows affected.


escalador.nombre,escalador.pais,num_ascensos,grado_medio,grado_maximo
Raúl Crespo,ESP,18,6a+/6b,7b
Chaken Gómez conde,ESP,16,7c+,8b
Adrian Alameda,ESP,14,6c+,8a+
Alex Garriga,ESP,13,7b+/7c,8a+
Xavier Gatell Romero,ESP,12,7b+,8a
jose luis palao,ESP,12,8a+/8b,8b+
Jose Agustí,ESP,10,7c+,8a
Gonzalo Larrocha,ESP,10,8a+/8b,9a
Tomata Tomata,ESP,10,6c+,7a+
luis rodriguez martin,ESP,10,8a/+,8b+


### 6.a) Dificultad media y maxima de los ascensos NO "Top Rope" de los escaladores con menos de 3 años de experiencia

Por desgracia no he encontrado funciones nativas de cypher para el tratamiento de las fechas, y he escrito el año actual hard-coded. Una opcion para ello es el uso de la libreria APOC https://github.com/neo4j-contrib/neo4j-apoc-procedures pero requiere preinstalación...

In [21]:
%%cypher 

MATCH (escalador:Escalador)
WHERE (2017 - escalador.anio_comienzo)>=0 AND
      (2017 - escalador.anio_comienzo)<3
WITH escalador

MATCH (escalador:Escalador)-[:REALIZA]-(ascenso:Ascenso)-[:TIENE]-(dificultad:Dificultad)
WHERE ascenso.tipo_encadenamiento<>'Toprope'
WITH count(ascenso) as num_ascensos, round(avg(dificultad.valor)) as dif_valor_medio, max(dificultad.valor) as dif_valor_max

MATCH (dif_media:Dificultad{valor:dif_valor_medio}), (dif_max:Dificultad{valor:dif_valor_max})
RETURN num_ascensos,
       dif_media.grado_frances as grado_medio,
       dif_max.grado_frances as grado_maximo


1 rows affected.


num_ascensos,grado_medio,grado_maximo
590,6a/+,8a


### 6.b) Dificultad media y maxima de los ascensos NO "Top Rope" de los escaladores con entre 10 y 30 años de experiencia

In [22]:
%%cypher 

MATCH (escalador:Escalador)
WHERE (2017 - escalador.anio_comienzo)>=10 AND
      (2017 - escalador.anio_comienzo)<30
WITH escalador

MATCH (escalador:Escalador)-[:REALIZA]-(ascenso:Ascenso)-[:TIENE]-(dificultad:Dificultad)
WHERE ascenso.tipo_encadenamiento<>'Toprope'
WITH count(ascenso) as num_ascensos, round(avg(dificultad.valor)) as dif_valor_medio, max(dificultad.valor) as dif_valor_max

MATCH (dif_media:Dificultad{valor:dif_valor_medio}), (dif_max:Dificultad{valor:dif_valor_max})
RETURN num_ascensos,
       dif_media.grado_frances as grado_medio,
       dif_max.grado_frances as grado_maximo


1 rows affected.


num_ascensos,grado_medio,grado_maximo
3818,7a,9c


### 7. Los 10 riscos españoles (o zonas) con mas ascensos por orden decreciente de numero de ascensos

In [30]:
%%cypher
MATCH (ascenso:Ascenso)
WHERE ascenso.pais ='ESP'
RETURN ascenso.risco, count(ascenso.risco) as num_ascensos
ORDER BY num_ascensos DESC
LIMIT 10

10 rows affected.


ascenso.risco,num_ascensos
MARGALEF,194
CUENCA,148
SIURANA,147
ALBARRACíN,137
RODELLAR,136
CHULILLA,134
MALLORCA,63
LA PEDRIZA,61
MONTSERRAT,58
EL CHORRO,44


### 8.a) Los 10 sectores españoles con mayor nivel de difcultad media de ascensos ordenadas por orden decreciente de dificultad y por numero de ascensos decreciente

In [38]:
%%cypher

MATCH (ascenso:Ascenso)-[:TIENE]-(dif:Dificultad)
WHERE ascenso.pais='ESP'
WITH ascenso.risco as nombre_risco, round(avg(dif.valor)) as dif_valor_medio, count(ascenso.risco) as num_ascensos
ORDER BY dif_valor_medio DESC,num_ascensos DESC, ascenso.risco ASC
LIMIT 10

MATCH (dif_media:Dificultad{valor:dif_valor_medio})
RETURN nombre_risco, dif_valor_medio, dif_media.grado_frances as grado_medio, num_ascensos


10 rows affected.


nombre_risco,dif_valor_medio,grado_medio,num_ascensos
LA COVA DE L'OCELL,71.0,8c/+,2
POLORIA,66.0,8b,2
FIGOLS,66.0,8b,1
ORIHUELA,65.0,8a+/8b,2
SANTA LINYA,64.0,8a+,18
OLIANA,64.0,8a+,14
FORADA,64.0,8a+,3
LA MUELA,64.0,8a+,2
BALTZOLA,64.0,8a+,1
CACIN,64.0,8a+,1


### 8.b) Las 10 sectores españoles con menor nivel de dificultad medio de ascensos ordenadas por orden creciente de dificultad y por numero de ascensos decreciente

In [39]:
%%cypher

MATCH (ascenso:Ascenso)-[:TIENE]-(dif:Dificultad)
WHERE ascenso.pais='ESP'
WITH ascenso.risco as nombre_risco, round(avg(dif.valor)) as dif_valor_medio, count(ascenso.risco) as num_ascensos
ORDER BY dif_valor_medio ASC,num_ascensos DESC, ascenso.risco ASC
LIMIT 10

MATCH (dif_media:Dificultad{valor:dif_valor_medio})
RETURN nombre_risco, dif_valor_medio, dif_media.grado_frances as grado_medio, num_ascensos


10 rows affected.


nombre_risco,dif_valor_medio,grado_medio,num_ascensos
PORTOMARIN,21.0,4a,2
MONT-RAL,25.0,4c,1
OTO,25.0,4c,1
PUERTO ROQUE,26.0,4c+,3
PUEBLA DE GUZMAN,27.0,4+,5
BALNEARIO DE PANTICOSA,27.0,4+,2
SAVASSONA,29.0,5a,2
LA DEHESA DE LA OLIVA,29.0,5a,1
LA PUENTE,29.0,5a,1
MALLORKA,29.0,5a,1
